<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt